In [1]:

import os
import requests
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup
import time

def download(url: str, fname: str, chunk_size=1024):
    resp = requests.get(url, stream=True)
    print(f'url:{url}')
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
        desc=fname,
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
            size = file.write(data)
            bar.update(size)


def fetch_html(url:str):
    resp = requests.get(url)
    resp.raise_for_status()
    
    return resp.content


if not os.path.exists('../data/download'):
    os.makedirs('../data/download')

/tmp/ipykernel_29966/306174593.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Extract data source 1

In [2]:
link1 = 'https://nyc3.digitaloceanspaces.com/owid-public/data/co2/owid-co2-data.csv'
download(link1,'../data/download/data1.csv')

url:https://nyc3.digitaloceanspaces.com/owid-public/data/co2/owid-co2-data.csv


../data/download/data1.csv: 100%|██████████| 12.5M/12.5M [00:04<00:00, 2.80MiB/s]


### Data 1 transformation

In [3]:
df1 = pd.read_csv('../data/download/data1.csv')
df1.info()
df1 = df1[df1['country'] == 'World']
df1 = df1[df1['co2'].notna()][['year','co2','co2_growth_prct']]
df1.iloc[0,2] = 0
df1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48058 entries, 0 to 48057
Data columns (total 79 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   country                                    48058 non-null  object 
 1   year                                       48058 non-null  int64  
 2   iso_code                                   39717 non-null  object 
 3   population                                 39495 non-null  float64
 4   gdp                                        14564 non-null  float64
 5   cement_co2                                 23764 non-null  float64
 6   cement_co2_per_capita                      22017 non-null  float64
 7   co2                                        30308 non-null  float64
 8   co2_growth_abs                             28157 non-null  float64
 9   co2_growth_prct                            25136 non-null  float64
 10  co2_including_luc     

,year,co2,co2_growth_prct
47266,1750,9.306,0.000
47267,1751,9.407,1.088
47268,1752,9.505,1.041
47269,1753,9.610,1.108
47270,1754,9.734,1.281
...,...,...,...
47534,2018,36766.945,2.058
47535,2019,37040.102,0.743
47536,2020,35007.738,-5.487
47537,2021,36816.543,5.167


### Load data source 1

In [4]:

df1.to_csv('../data/co2_emission.csv')

### Extract data source 2

In [5]:
link2 = 'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/air_temperature_mean/'
content = fetch_html(link2)

soup = BeautifulSoup(content,'html.parser')

In [6]:
csv_links = [(a.text,f'{link2}{a["href"]}') for a in soup.find_all('a') if a.text != '../']
csv_links

[('regional_averages_tm_01.txt',
  'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/air_temperature_mean/regional_averages_tm_01.txt'),
 ('regional_averages_tm_02.txt',
  'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/air_temperature_mean/regional_averages_tm_02.txt'),
 ('regional_averages_tm_03.txt',
  'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/air_temperature_mean/regional_averages_tm_03.txt'),
 ('regional_averages_tm_04.txt',
  'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/air_temperature_mean/regional_averages_tm_04.txt'),
 ('regional_averages_tm_05.txt',
  'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/air_temperature_mean/regional_averages_tm_05.txt'),
 ('regional_averages_tm_06.txt',
  'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/air_temperature_mean/regional_averages_tm_06.txt'),
 ('regiona

In [7]:
for name,link in csv_links:
    download(link,f'../data/download/{name}')

url:https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/air_temperature_mean/regional_averages_tm_01.txt


../data/download/regional_averages_tm_01.txt: 100%|██████████| 25.7k/25.7k [00:00<00:00, 3.80MiB/s]


url:https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/air_temperature_mean/regional_averages_tm_02.txt


../data/download/regional_averages_tm_02.txt: 100%|██████████| 25.7k/25.7k [00:00<00:00, 2.35MiB/s]


url:https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/air_temperature_mean/regional_averages_tm_03.txt


...

../data/download/regional_averages_tm_10.txt: 100%|██████████| 25.5k/25.5k [00:00<00:00, 771kiB/s]


url:https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/air_temperature_mean/regional_averages_tm_11.txt


../data/download/regional_averages_tm_11.txt: 100%|██████████| 25.5k/25.5k [00:00<00:00, 483kiB/s]


url:https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/air_temperature_mean/regional_averages_tm_12.txt


../data/download/regional_averages_tm_12.txt: 100%|██████████| 25.5k/25.5k [00:00<00:00, 1.89MiB/s]


### Data 2 transform

In [8]:
df2_list = []

for i,(name,_) in enumerate(csv_links):
    df2_list.append(pd.read_csv(f'../data/download/{name}',skiprows=1,delimiter=';'))
    df2_list[i].drop(df2_list[i].columns[-1], axis=1, inplace=True)

df2_list[4]

,Jahr,Monat,Brandenburg/Berlin,Brandenburg,Baden-Wuerttemberg,Bayern,Hessen,Mecklenburg-Vorpommern,Niedersachsen,Niedersachsen/Hamburg/Bremen,Nordrhein-Westfalen,Rheinland-Pfalz,Schleswig-Holstein,Saarland,Sachsen,Sachsen-Anhalt,Thueringen/Sachsen-Anhalt,Thueringen,Deutschland
0,1881,5,12.85,12.83,11.44,10.99,12.15,11.58,12.03,12.03,12.55,12.25,11.16,12.38,11.49,12.57,12.05,11.40,11.81
1,1882,5,12.21,12.19,12.11,11.63,11.87,11.44,12.26,12.26,12.53,12.30,11.88,12.63,11.34,12.32,11.82,11.19,11.96
2,1883,5,12.84,12.83,12.55,11.99,12.36,11.89,12.45,12.45,12.87,12.62,11.96,13.12,11.94,12.95,12.55,12.06,12.38
3,1884,5,13.32,13.31,12.96,12.41,12.61,12.12,12.56,12.56,12.99,12.91,11.79,13.23,12.68,13.20,12.73,12.15,12.67
4,1885,5,11.28,11.27,9.70,9.59,9.66,9.87,9.79,9.79,9.88,9.82,9.38,9.94,10.65,10.73,10.25,9.65,9.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2019,5,12.30,12.28,10.44,10.24,10.87,11.33,11.30,11.30,11.08,11.13,10.77,11.45,10.97,11.83,11.21,10.40,10.99
139,2020,5,12.19,12.18,12.37,11.58,11.98,11.51,11.87,11.87,12.42,12.82,11.21,13.33,11.30,12.14,11.65,11.03,11.91
140,2021,5,11.93,11.92,10.13,10.05,10.48,11.11,10.94,10.94,10.96,10.49,10.50,10.54,10.95,11.71,11.12,10.34,10.72
141,2022,5,14.92,14.90,14.88,14.53,14.73,13.40,13.93,13.92,14.58,15.27,12.76,15.59,14.60,14.92,14.58,14.13,14.44


In [9]:
df2 = pd.concat(df2_list, axis=0).sort_values(by=['Jahr','Monat'])
df2 = df2.dropna()
df2.info()
df2

<class 'pandas.core.frame.DataFrame'>
Index: 1720 entries, 0 to 143
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Jahr                          1720 non-null   int64  
 1   Monat                         1720 non-null   int64  
 2   Brandenburg/Berlin            1720 non-null   float64
 3   Brandenburg                   1720 non-null   float64
 4   Baden-Wuerttemberg            1720 non-null   float64
 5   Bayern                        1720 non-null   float64
 6   Hessen                        1720 non-null   float64
 7   Mecklenburg-Vorpommern        1720 non-null   float64
 8   Niedersachsen                 1720 non-null   float64
 9   Niedersachsen/Hamburg/Bremen  1720 non-null   float64
 10  Nordrhein-Westfalen           1720 non-null   float64
 11  Rheinland-Pfalz               1720 non-null   float64
 12  Schleswig-Holstein            1720 non-null   float64
 13  Saarland 

,Jahr,Monat,Brandenburg/Berlin,Brandenburg,Baden-Wuerttemberg,Bayern,Hessen,Mecklenburg-Vorpommern,Niedersachsen,Niedersachsen/Hamburg/Bremen,Nordrhein-Westfalen,Rheinland-Pfalz,Schleswig-Holstein,Saarland,Sachsen,Sachsen-Anhalt,Thueringen/Sachsen-Anhalt,Thueringen,Deutschland
0,1881,1,-5.54,-5.56,-4.89,-6.51,-5.68,-5.07,-4.55,-4.55,-4.21,-4.49,-4.06,-4.15,-6.22,-5.89,-6.28,-6.76,-5.36
0,1881,2,-1.00,-1.01,1.33,-0.73,0.75,-2.01,-0.05,-0.07,1.64,1.45,-1.81,2.25,-0.92,-0.42,-0.34,-0.25,-0.10
0,1881,3,1.71,1.70,4.42,2.50,3.43,0.40,2.20,2.19,3.81,4.49,0.54,5.37,1.79,2.20,2.18,2.14,2.61
0,1881,4,5.56,5.55,6.63,5.01,5.88,4.63,5.64,5.64,6.44,6.65,5.02,7.08,4.41,5.51,5.04,4.43,5.55
0,1881,5,12.85,12.83,11.44,10.99,12.15,11.58,12.03,12.03,12.55,12.25,11.16,12.38,11.49,12.57,12.05,11.40,11.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,2023,12,3.81,3.80,3.89,2.83,4.27,3.60,4.98,4.97,5.56,4.71,4.13,4.84,3.67,4.41,4.08,3.65,4.05
143,2024,1,1.34,1.34,1.32,0.40,1.29,1.88,2.53,2.53,2.64,1.63,2.46,1.90,0.82,1.68,1.24,0.66,1.50
143,2024,2,6.65,6.64,6.54,6.09,6.72,5.78,6.95,6.94,7.48,7.01,5.96,7.01,6.44,6.97,6.64,6.22,6.58
143,2024,3,7.71,7.69,7.39,7.13,7.33,7.00,7.79,7.79,8.34,7.97,6.98,8.26,7.32,7.60,7.24,6.77,7.48


### Load data 2

In [10]:
df2.to_csv('../data/temperature.csv')

### Extract data source 3

In [11]:
link3 = 'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/precipitation/'
content = fetch_html(link3)

soup = BeautifulSoup(content,'html.parser')

In [12]:
csv_links = [(a.text,f'{link3}{a["href"]}') for a in soup.find_all('a') if a.text != '../']
csv_links

[('regional_averages_rr_01.txt',
  'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/precipitation/regional_averages_rr_01.txt'),
 ('regional_averages_rr_02.txt',
  'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/precipitation/regional_averages_rr_02.txt'),
 ('regional_averages_rr_03.txt',
  'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/precipitation/regional_averages_rr_03.txt'),
 ('regional_averages_rr_04.txt',
  'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/precipitation/regional_averages_rr_04.txt'),
 ('regional_averages_rr_05.txt',
  'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/precipitation/regional_averages_rr_05.txt'),
 ('regional_averages_rr_06.txt',
  'https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/precipitation/regional_averages_rr_06.txt'),
 ('regional_averages_rr_07.txt',
  'https://opendata

In [13]:
for name,link in csv_links:
    download(link,f'../data/download/{name}')

url:https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/precipitation/regional_averages_rr_01.txt


../data/download/regional_averages_rr_01.txt: 100%|██████████| 23.3k/23.3k [00:00<00:00, 361kiB/s]


url:https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/precipitation/regional_averages_rr_02.txt


../data/download/regional_averages_rr_02.txt: 100%|██████████| 23.3k/23.3k [00:00<00:00, 767kiB/s]


url:https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/precipitation/regional_averages_rr_03.txt


...

../data/download/regional_averages_rr_10.txt: 100%|██████████| 23.1k/23.1k [00:00<00:00, 1.48MiB/s]


url:https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/precipitation/regional_averages_rr_11.txt


../data/download/regional_averages_rr_11.txt: 100%|██████████| 23.1k/23.1k [00:00<00:00, 1.13MiB/s]


url:https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/monthly/precipitation/regional_averages_rr_12.txt


../data/download/regional_averages_rr_12.txt: 100%|██████████| 23.1k/23.1k [00:00<00:00, 1.92MiB/s]


### Data source 3 transform

In [14]:
df3_list = []

for i,(name,_) in enumerate(csv_links):
    df3_list.append(pd.read_csv(f'../data/download/{name}',skiprows=1,delimiter=';'))
    df3_list[i].drop(df3_list[i].columns[-1], axis=1, inplace=True)

df3_list[4]

,Jahr,Monat,Brandenburg/Berlin,Brandenburg,Baden-Wuerttemberg,Bayern,Hessen,Mecklenburg-Vorpommern,Niedersachsen,Niedersachsen/Hamburg/Bremen,Nordrhein-Westfalen,Rheinland-Pfalz,Schleswig-Holstein,Saarland,Sachsen,Sachsen-Anhalt,Thueringen/Sachsen-Anhalt,Thueringen,Deutschland
0,1881,5,31.3,31.3,61.5,95.0,31.1,16.3,25.8,25.8,31.9,38.0,30.0,36.7,57.9,30.4,33.8,38.0,47.3
1,1882,5,48.9,48.9,86.1,77.0,57.8,33.2,39.7,39.8,54.1,63.1,43.0,65.4,77.0,44.2,49.4,55.9,59.1
2,1883,5,46.0,46.0,75.5,69.5,47.2,40.4,40.5,40.4,35.3,38.0,37.3,40.5,51.7,48.6,50.8,53.5,51.5
3,1884,5,51.4,51.3,61.6,52.2,67.1,55.5,57.2,57.2,69.2,65.9,61.7,69.5,48.8,48.0,50.0,52.6,57.4
4,1885,5,64.8,64.7,111.0,96.7,73.3,64.8,80.9,80.9,83.3,76.9,78.9,84.4,72.7,66.7,68.4,70.6,82.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2019,5,39.2,39.3,121.3,130.8,99.3,37.5,42.8,42.9,62.7,79.7,55.7,89.1,63.3,43.1,60.9,84.1,78.6
139,2020,5,32.3,32.3,53.0,58.5,36.2,22.7,24.1,24.1,18.4,31.9,27.6,44.9,54.4,30.1,38.4,49.3,38.4
140,2021,5,63.9,64.2,124.1,127.9,79.6,63.7,87.7,87.8,89.8,84.6,104.2,105.4,94.8,64.7,72.2,81.9,94.9
141,2022,5,28.5,28.5,48.6,66.6,38.5,34.7,47.5,47.6,47.2,39.8,71.5,45.2,33.3,34.0,35.0,36.4,47.0


In [15]:
df3 = pd.concat(df3_list, axis=0).sort_values(by=['Jahr','Monat'])
df3 = df3.dropna()
df3.info()
df3

<class 'pandas.core.frame.DataFrame'>
Index: 1720 entries, 0 to 143
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Jahr                          1720 non-null   int64  
 1   Monat                         1720 non-null   int64  
 2   Brandenburg/Berlin            1720 non-null   float64
 3   Brandenburg                   1720 non-null   float64
 4   Baden-Wuerttemberg            1720 non-null   float64
 5   Bayern                        1720 non-null   float64
 6   Hessen                        1720 non-null   float64
 7   Mecklenburg-Vorpommern        1720 non-null   float64
 8   Niedersachsen                 1720 non-null   float64
 9   Niedersachsen/Hamburg/Bremen  1720 non-null   float64
 10  Nordrhein-Westfalen           1720 non-null   float64
 11  Rheinland-Pfalz               1720 non-null   float64
 12  Schleswig-Holstein            1720 non-null   float64
 13  Saarland 

,Jahr,Monat,Brandenburg/Berlin,Brandenburg,Baden-Wuerttemberg,Bayern,Hessen,Mecklenburg-Vorpommern,Niedersachsen,Niedersachsen/Hamburg/Bremen,Nordrhein-Westfalen,Rheinland-Pfalz,Schleswig-Holstein,Saarland,Sachsen,Sachsen-Anhalt,Thueringen/Sachsen-Anhalt,Thueringen,Deutschland
0,1881,1,20.4,20.3,36.2,22.5,33.2,16.8,35.1,34.9,40.6,39.3,21.5,52.8,21.8,23.5,25.0,26.8,28.7
0,1881,2,24.2,24.1,56.7,39.4,56.0,21.7,43.9,43.8,68.2,63.7,30.5,77.0,34.7,29.0,36.5,46.0,43.7
0,1881,3,60.7,60.7,84.6,89.0,99.5,43.2,91.3,90.9,124.7,95.4,55.4,101.4,101.5,74.4,84.6,97.5,86.6
0,1881,4,8.8,8.8,64.6,52.4,26.2,5.2,11.9,11.8,28.4,41.9,7.7,48.4,26.3,11.1,18.3,27.4,30.1
0,1881,5,31.3,31.3,61.5,95.0,31.1,16.3,25.8,25.8,31.9,38.0,30.0,36.7,57.9,30.4,33.8,38.0,47.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,2023,12,99.7,99.2,113.9,125.0,104.2,88.5,158.6,158.3,160.7,88.0,119.2,118.5,106.8,101.3,102.6,104.3,120.4
143,2024,1,47.0,47.0,82.1,85.6,79.1,62.4,82.2,82.3,85.3,88.8,90.1,123.8,51.0,45.6,54.2,65.6,75.2
143,2024,2,73.8,73.7,74.4,69.6,86.2,71.2,94.6,94.5,103.8,81.2,93.4,115.2,80.5,67.8,72.9,79.6,81.1
143,2024,3,28.7,28.6,71.5,48.1,52.7,18.8,44.4,44.2,66.2,69.3,30.7,92.7,18.0,29.6,31.1,33.1,45.8


### Load Data source 3

In [16]:
df3.to_csv('../data/precipitation.csv')

### Extract Data source 4

In [17]:
link4 = 'https://opendata.dwd.de/climate_environment/CDC/derived_germany/soil/monthly/historical'
content = fetch_html(link4)

soup = BeautifulSoup(content,'html.parser')

files = [a['href'] for a in soup.find_all('a') if a['href'].endswith('.gz') and 'v2' in a['href']]
files

['derived_germany_soil_monthly_historical_v2_1001.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1048.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1050.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1052.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1072.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1078.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1103.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1107.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1161.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1197.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1200.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1207.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1214.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1224.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1246.txt.gz',
 'derived_germany_soil_monthly_historical_v2_125.txt.gz',
 'derived_germany_soil_monthly_historical_v2_1255.txt.gz'

In [18]:
for file in tqdm(files):
    time.sleep(1)
    download(f'{link4}/{file}',f'../data/download/{file}')

  0%|          | 0/486 [00:00<?, ?it/s]

url:https://opendata.dwd.de/climate_environment/CDC/derived_germany/soil/monthly/historical/derived_germany_soil_monthly_historical_v2_1001.txt.gz


../data/download/derived_germany_soil_monthly_historical_v2_1001.txt.gz: 100%|██████████| 23.8k/23.8k [00:00<00:00, 9.35MiB/s]
  0%|          | 1/486 [00:01<10:05,  1.25s/it]

url:https://opendata.dwd.de/climate_environment/CDC/derived_germany/soil/monthly/historical/derived_germany_soil_monthly_historical_v2_1048.txt.gz


../data/download/derived_germany_soil_monthly_historical_v2_1048.txt.gz: 100%|██████████| 23.8k/23.8k [00:00<00:00, 621kiB/s]
  0%|          | 2/486 [00:02<10:29,  1.30s/it]

...

../data/download/derived_germany_soil_monthly_historical_v2_963.txt.gz: 100%|██████████| 23.3k/23.3k [00:00<00:00, 1.32MiB/s]
100%|█████████▉| 484/486 [12:55<00:02,  1.32s/it]

url:https://opendata.dwd.de/climate_environment/CDC/derived_germany/soil/monthly/historical/derived_germany_soil_monthly_historical_v2_983.txt.gz


../data/download/derived_germany_soil_monthly_historical_v2_983.txt.gz: 100%|██████████| 12.5k/12.5k [00:00<00:00, 8.93MiB/s]
100%|█████████▉| 485/486 [12:56<00:01,  1.30s/it]

url:https://opendata.dwd.de/climate_environment/CDC/derived_germany/soil/monthly/historical/derived_germany_soil_monthly_historical_v2_991.txt.gz


../data/download/derived_germany_soil_monthly_historical_v2_991.txt.gz: 100%|██████████| 23.4k/23.4k [00:00<00:00, 1.15MiB/s]
100%|██████████| 486/486 [12:57<00:00,  1.60s/it]


### Data source 4 transform

In [19]:
df4_list = []

for file in tqdm(files):
    df = pd.read_csv(f'../data/download/{file}',delimiter=';')
    df.drop(df.columns[-2:], axis=1, inplace=True)
    df4_list.append(df)

df4_list[0]

100%|██████████| 486/486 [00:04<00:00, 107.80it/s]


,Stationsindex,Monat,Mittel von TS05,Mittel von TS10,Mittel von TS20,Mittel von TS50,Mittel von TS100,Mittel von TSLS05,Mittel von TSSL05,Maximum von ZFUMI,...,Mittel von BFMS_AG,Mittel von BFML_AG,Summe von VPGFAO,Summe von VPGH,Summe von VRGS_AG,Summe von VRGL_AG,Summe von VRWS_AG,Summe von VRWL_AG,Summe von VRMS_AG,Summe von VRML_AG
0,1001,199101,1.0,1.4,1.8,2.9,4.3,0.8,1.1,24.0,...,90.1,102.9,22.6,18.3,9.5,9.5,8.0,9.6,7.8,9.7
1,1001,199102,-1.9,-1.5,-0.5,1.0,2.7,-2.3,-1.9,57.0,...,74.0,99.9,16.8,11.3,6.7,6.7,6.2,8.9,5.5,9.3
2,1001,199103,6.1,5.8,5.2,4.5,4.3,6.1,6.1,52.0,...,69.3,98.1,41.5,34.0,20.3,20.4,16.7,21.9,12.9,19.9
3,1001,199104,9.1,9.0,8.7,8.1,7.3,9.0,9.0,0.0,...,71.1,98.4,61.5,56.9,37.3,37.5,36.6,42.4,20.3,29.7
4,1001,199105,12.0,11.8,11.3,10.3,9.1,11.9,11.8,0.0,...,83.5,105.3,80.9,61.4,50.8,50.8,82.8,85.9,31.8,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,1001,202308,22.3,22.2,21.9,20.9,19.3,22.1,22.1,0.0,...,10.9,38.7,92.2,107.1,53.8,65.3,31.0,36.9,73.1,102.0
392,1001,202309,20.3,20.2,20.1,19.7,19.0,20.0,20.1,0.0,...,7.5,25.8,75.1,128.5,30.3,41.7,22.1,25.9,23.9,32.5
393,1001,202310,12.9,13.2,13.6,14.6,15.8,12.9,12.9,0.0,...,48.2,45.3,37.0,41.9,21.8,22.4,14.5,16.0,12.6,14.9
394,1001,202311,5.9,6.3,7.0,9.0,11.2,5.9,6.0,0.0,...,103.1,91.9,19.6,17.6,15.1,15.1,9.3,10.1,9.0,10.1


In [20]:
df4 = pd.concat(df4_list).dropna()
df4['Jahr'] = df4['Monat'] // 100
df4['Monat'] = df4['Monat'] % 100
df4 = df4.sort_values(by=['Stationsindex','Jahr','Monat'])
df4.info()
df4

<class 'pandas.core.frame.DataFrame'>
Index: 166318 entries, 0 to 11
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Stationsindex         166318 non-null  int64  
 1   Monat                 166318 non-null  int64  
 2   Mittel von TS05       166318 non-null  float64
 3   Mittel von TS10       166318 non-null  float64
 4   Mittel von TS20       166318 non-null  float64
 5   Mittel von TS50       166318 non-null  float64
 6   Mittel von TS100      166318 non-null  float64
 7   Mittel von TSLS05     166318 non-null  float64
 8   Mittel von TSSL05     166318 non-null  float64
 9   Maximum von ZFUMI     166318 non-null  float64
 10  Maximum von ZTKMI     166318 non-null  float64
 11  Maximum von ZTUMI     166318 non-null  float64
 12  Mittel von BFGL01_AG  166318 non-null  float64
 13  Mittel von BFGL02_AG  166318 non-null  float64
 14  Mittel von BFGL03_AG  166318 non-null  float64
 15  Mittel vo

,Stationsindex,Monat,Mittel von TS05,Mittel von TS10,Mittel von TS20,Mittel von TS50,Mittel von TS100,Mittel von TSLS05,Mittel von TSSL05,Maximum von ZFUMI,...,Mittel von BFML_AG,Summe von VPGFAO,Summe von VPGH,Summe von VRGS_AG,Summe von VRGL_AG,Summe von VRWS_AG,Summe von VRWL_AG,Summe von VRMS_AG,Summe von VRML_AG,Jahr
0,44,1,1.6,1.9,2.3,3.4,4.8,1.7,1.9,16.0,...,117.8,18.5,11.6,13.0,12.7,6.5,7.3,5.9,6.8,1991
1,44,2,-1.6,-1.4,-0.8,0.8,2.7,-1.5,-1.2,50.0,...,109.6,15.9,10.0,5.2,5.2,5.4,7.2,5.2,7.5,1991
2,44,3,7.0,6.8,6.4,5.5,4.8,6.9,6.8,30.0,...,106.0,40.2,34.7,20.2,20.2,18.0,23.2,11.6,18.9,1991
3,44,4,9.5,9.4,9.2,8.5,7.7,9.4,9.5,0.0,...,99.5,64.1,61.5,35.0,35.6,39.8,45.4,18.4,28.6,1991
4,44,5,12.2,12.0,11.6,10.6,9.4,12.1,12.1,0.0,...,104.6,70.0,52.7,43.0,43.1,70.9,73.5,23.5,32.8,1991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,19207,8,21.3,21.2,20.9,19.8,18.3,20.7,20.8,0.0,...,67.5,84.9,82.3,65.3,65.3,38.1,45.4,94.9,104.3,2023
8,19207,9,20.4,20.3,20.2,19.6,18.7,19.8,19.8,0.0,...,44.6,66.5,94.2,41.2,51.2,23.4,29.3,30.4,46.8,2023
9,19207,10,11.8,12.1,12.6,14.0,15.2,11.5,11.7,0.0,...,69.9,27.2,26.2,22.2,23.0,12.7,14.4,11.2,13.4,2023
10,19207,11,5.7,6.1,6.8,8.7,10.7,5.5,5.8,5.0,...,107.0,14.8,13.1,11.4,11.4,6.3,7.0,6.2,6.7,2023


### Load Data source 4

In [21]:
df4.to_csv('../data/soil_condition.csv')

#### Extract data source 5

In [22]:
link5 = 'https://opendata.dwd.de/climate_environment/CDC/derived_germany/soil/monthly/historical/derived_germany_soil_monthly_historical_stations_list.txt'
download(link5,'../data/download/soil_stations_list.csv')

url:https://opendata.dwd.de/climate_environment/CDC/derived_germany/soil/monthly/historical/derived_germany_soil_monthly_historical_stations_list.txt


../data/download/soil_stations_list.csv: 100%|██████████| 96.3k/96.3k [00:00<00:00, 1.29MiB/s]


### Data source 5 transform

In [23]:
df5 = pd.read_csv('../data/download/soil_stations_list.csv',delimiter=';',encoding='ISO 8859-15')
df5.info()
df5

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487 entries, 0 to 486
Data columns (total 6 columns):
 #   Column                                                                            Non-Null Count  Dtype  
---  ------                                                                            --------------  -----  
 0   Stationsindex                                                                     487 non-null    int64  
 1    Höhe in m                                                                        487 non-null    int64  
 2   Breite                                                                            487 non-null    float64
 3   Länge                                                                             487 non-null    float64
 4   Name                                                                              487 non-null    object 
 5   Bundesland                                                                        487 non-null    object 
dtypes: 

,Stationsindex,Höhe in m,Breite,Länge,Name,Bundesland
0,44,44,52.93,8.24,Großenkneten ...,Niedersachsen ...
1,73,374,48.62,13.06,Aldersbach-Kramersepp ...,Bayern ...
2,78,64,52.49,7.91,Alfhausen ...,Niedersachsen ...
3,91,300,50.75,9.35,Alsfeld-Eifa ...,Hessen ...
4,96,50,52.94,12.85,Neuruppin-Alt Ruppin ...,Brandenburg ...
...,...,...,...,...,...,...
482,15555,815,47.88,10.59,Kaufbeuren-Oberbeuren ...,Bayern ...
483,15813,40,52.51,7.41,Lingen-Baccum ...,Niedersachsen ...
484,19171,13,54.00,9.86,Hasenkrug-Hardebek ...,Schleswig-Holstein ...
485,19172,48,54.03,9.39,Wacken ...,Schleswig-Holstein ...


In [24]:
df5 =df5.iloc[:, [0,4,5]]
df5.iloc[:, 1] = df5.iloc[:, 1].apply(lambda x: x.strip())
df5.iloc[:, 2] = df5.iloc[:, 2].apply(lambda x: x.strip())
df5 = df5.dropna()
df5.info()
df5

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487 entries, 0 to 486
Data columns (total 3 columns):
 #   Column                                                                            Non-Null Count  Dtype 
---  ------                                                                            --------------  ----- 
 0   Stationsindex                                                                     487 non-null    int64 
 1   Name                                                                              487 non-null    object
 2   Bundesland                                                                        487 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.5+ KB


,Stationsindex,Name,Bundesland
0,44,Großenkneten,Niedersachsen
1,73,Aldersbach-Kramersepp,Bayern
2,78,Alfhausen,Niedersachsen
3,91,Alsfeld-Eifa,Hessen
4,96,Neuruppin-Alt Ruppin,Brandenburg
...,...,...,...
482,15555,Kaufbeuren-Oberbeuren,Bayern
483,15813,Lingen-Baccum,Niedersachsen
484,19171,Hasenkrug-Hardebek,Schleswig-Holstein
485,19172,Wacken,Schleswig-Holstein


### Load data 5

In [25]:
df5.to_csv('../data/station.csv',encoding='UTF-8')